## Installing the envinroment

In [ ]:
# CONDA_SUBDIR=osx-64 conda create -n project1
# conda activate project1
# conda config --env --set subdir osx-64
# conda install -c bioconda fastqc
# conda install -c bioconda snpeff

## **Downloading and unpacking the data**

First, we need the reference sequence of the parental (unevolved, not resistant to antibiotics) E. coli strain.
We need sequence in fasta format (GCF_000005845.2_ASM584v2_genomic.fna.gz) and annotation in .gff format(*_genomic.gff.gz). This is E.coli strain K-12 substrain MG1655, laboratory workhorse.

In [ ]:
! wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz

In [ ]:
! gzip -d GCF_000005845.2_ASM584v2_genomic.fna.gz

Then we will need raw Illumina sequencing reads from shotgun sequencing of an E. coli strain that is resistant to the antibiotic ampicillin:
https://doi.org/10.6084/m9.figshare.10006541.v3

In [ ]:
! wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gff.gz

In [ ]:
! gzip -d GCF_000005845.2_ASM584v2_genomic.gff.gz

## Inspect raw sequencing data manually

Let's check how many reads are in each fastq file. Some reads will get removed during analysis, so it’s important to know what we started with.

In [ ]:
! wc -l amp_res_1.fastq
# 1823504 amp_res_1.fastq

In [3]:
1823504 // 4

455876

In [ ]:
! wc -l amp_res_2.fastq
# 1823504 amp_res_2.fastq

In [4]:
1823504 // 4

455876

## Inspect raw sequencing data with FastQC.

Run the program fastqc on the two fastq files.<br>
We also have to tell fastqc where to put the output files (use  ‘-o . ’ to output files to the current directory).<br>
We have to specify the full root path to each fastq file.  

In [ ]:
! fastqc -o . ~/OneDrive/Study/BI/BioInf/Project\ \#1/raw_data/amp_res_1.fastq ~/OneDrive/Study/BI/BioInf/Project\ \#1/raw_data/amp_res_2.fastq
# 455876 in amp_res_1_fastqc.html
# 455876 in amp_res_2_fastqc.html

In [6]:
455876 * 4

1823504

Red circles in:<br>
1. Per base sequence quality in amp_res_1_fastqc.html
2. Per tile sequence quality in amp_res_1_fastqc.html
3. Per base sequence quality in amp_res_2_fastqc.html

## Filtering the reads.

Run Trimmomatic in paired end mode, with following parameters:<br>
●	Cut bases off the start of a read if quality below 20.<br>
●	Cut bases off the end of a read if quality below 20.<br>
●	Trim reads using a sliding window approach, with window size 10 and average quality  within the window 20.<br>
●	Drop the read if it is below length 20.


In [ ]:
! trimmomatic PE -phred33 amp_res_1.fastq amp_res_2.fastq output_forward_paired.fastq output_forward_unpaired.fastq output_reverse_paired.fastq output_reverse_unpaired.fastq LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20 MINLEN:20

In [ ]:
! wc -l output_forward_paired.fastq
   #1785036 output_forward_paired.fastq
! wc -l output_forward_unpaired.fastq
   #36864 output_forward_unpaired.fastq
! wc -l output_reverse_paired.fastq
   #1785036 output_reverse_paired.fastq
! wc -l output_reverse_unpaired.fastq
   #1092 output_reverse_unpaired.fastq

In [5]:
1785036 // 4

446259

Run the program fastqc on the two fastq files.<br>

In [ ]:
! fastqc -o . ~/OneDrive/Study/BI/BioInf/Project\ \#1/raw_data/output_forward_paired.fastq ~/OneDrive/Study/BI/BioInf/Project\ \#1/raw_data/output_reverse_paired.fastq

Red circles in:<br>
1. Per tile sequence quality in output_forward_paired_fastqc.html

### Let's test what happens if we increase the quality score at all steps to 30? 

Make a new directory and copy files into it.

In [ ]:
! mkdir test_trimmomatic_30

In [ ]:
! cp amp_res_1.fastq ~/OneDrive/Study/BI/BioInf/Project\ \#1/raw_data/test_trimmomatic_30

In [ ]:
! cp amp_res_2.fastq ~/OneDrive/Study/BI/BioInf/Project\ \#1/raw_data/test_trimmomatic_30

Run Trimmomatic in paired end mode, with following parameters:<br>
●	Cut bases off the start of a read if quality below 30.<br>
●	Cut bases off the end of a read if quality below 30.<br>
●	Trim reads using a sliding window approach, with window size 10 and average quality  within the window 30.<br>
●	Drop the read if it is below length 30.

In [ ]:
! trimmomatic PE -phred33 amp_res_1.fastq amp_res_2.fastq output_forward_paired_30.fastq output_forward_unpaired_30.fastq output_reverse_paired_30.fastq output_reverse_unpaired_30.fastq LEADING:30 TRAILING:30 SLIDINGWINDOW:10:30 MINLEN:30

In [ ]:
! wc -l output_forward_paired_30.fastq
  #1440836 output_forward_paired_30.fastq
! wc -l output_forward_unpaired_30.fastq
  #146864 output_forward_unpaired_30.fastq
! wc -l output_reverse_paired_30.fastq
  #1440836 output_reverse_paired_30.fastq
! wc -l output_reverse_unpaired_30.fastq
  #109164 output_reverse_unpaired_30.fastq

In [7]:
1440836 // 4

360209

In [ ]:
! fastqc -o . ~/OneDrive/Study/BI/BioInf/Project\ \#1/raw_data/test_trimmomatic_30/output_forward_paired_30.fastq ~/OneDrive/Study/BI/BioInf/Project\ \#1/raw_data/test_trimmomatic_30/output_reverse_paired_30.fastq

Red circles in:<br>
1. Per tile sequence quality in output_forward_paired_30_fastqc.html
2. Per tile sequence quality in output_reverse_paired_30_fastqc.html

That is why we decided to use results from trimmomatic with the quality score at all steps at 20

## Aligning sequences to reference

### Index the reference file 

We have to index the reference file. There is a command in bwa to do this.

In [ ]:
! bwa index GCF_000005845.2_ASM584v2_genomic.fna
#[bwa_index] Pack FASTA... 0.07 sec
#[bwa_index] Construct BWT for the packed sequence...
#[bwa_index] 1.01 seconds elapse.
#[bwa_index] Update BWT... 0.03 sec
#[bwa_index] Pack forward-only FASTA... 0.05 sec
#[bwa_index] Construct SA from BWT and Occ... 0.34 sec
#[main] Version: 0.7.17-r1188
#[main] CMD: bwa index GCF_000005845.2_ASM584v2_genomic.fna.gz
#[main] Real time: 1.492 sec; CPU: 1.793 sec

### Aligning reads

In [ ]:
! bwa mem GCF_000005845.2_ASM584v2_genomic.fna output_forward_paired.fastq output_reverse_paired.fastq > alignment.sam

### Compress SAM file

In [ ]:
! samtools view -S -b alignment.sam > alignment.bam

In [ ]:
! samtools flagstat alignment.bam
#892776 + 0 in total (QC-passed reads + QC-failed reads)
#892518 + 0 primary
#0 + 0 secondary
#258 + 0 supplementary
#0 + 0 duplicates
#0 + 0 primary duplicates
#891649 + 0 mapped (99.87% : N/A)
#891391 + 0 primary mapped (99.87% : N/A)
#892518 + 0 paired in sequencing
#446259 + 0 read1
#446259 + 0 read2
#888554 + 0 properly paired (99.56% : N/A)
#890412 + 0 with itself and mate mapped
#979 + 0 singletons (0.11% : N/A)
#0 + 0 with mate mapped to a different chr
#0 + 0 with mate mapped to a different chr (mapQ>=5)

99.87% of reads are mapped!

### Sort and index BAM file

In [ ]:
! samtools sort alignment.bam -o alignment_sorted.bam

In [ ]:
! samtools index alignment_sorted.bam

## Variant calling

First we need to make an intermediate file type called an mpileup, because it goes through each position and “piles up” the reads, tabulating the number of bases that match or don’t match the reference. 

In [ ]:
! samtools mpileup -f GCF_000005845.2_ASM584v2_genomic.fna alignment_sorted.bam > my.mpileup

To call actual variants, we will use a program called VarScan (variant scanner).

In [ ]:
! java -jar VarScan.v2.4.0.jar mpileup2snp my.mpileup --min-var-freq 0.70 --variants --output-vcf 1 > VarScan_results.vcf

## Automatic SNP annotation

### Step 0

To annotate vcf file, we need sequence and annotation of our reference, to build a custom database. Easiest way - use Genbank format that contains both annotation and sequence:<br>
**NB!!!!!! file is in GBFF format!!!!!**


In [ ]:
! wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gbff.gz

In [ ]:
! gzip -d GCF_000005845.2_ASM584v2_genomic.gbff.gz

### Step 1. Create database

#### Create empty text file snpEff.config, and add there just one string: k12.genome : ecoli_K12

In [ ]:
! echo k12.genome : ecoli_K12 > snpEff.config

#### Create folder for the database

In [ ]:
! mkdir -p data/k12

#### Put there .gbk file (rename to genes.gbk)

In [ ]:
! cp GCF_000005845.2_ASM584v2_genomic.gbff data/k12/genes.gbk

#### Create database

In [ ]:
! snpEff build -genbank -v k12

#### Annotate

In [ ]:
! snpEff ann k12 VarScan_results.vcf > VarScan_results_annotated.vcf

## Ending

Visualise VarScan_results_annotated.vcf in IGV or open it as a table in LibreOffice